In [2]:
import csv
import numpy as np
import pandas as pd
import json

In [14]:
KDs = pd.read_csv("Data/CFSKDsForVisualization.csv")
Modules = pd.read_csv("Data/CFSModulesForVisualization.csv")
Network = pd.read_csv("Data/CFSNetworkForVisualization_withoutNaN.csv")

### Modules of interest

1) NKM4

2) BcellM7

3) MonoM6

4) NKM13

5) NKM9

In [15]:
Modules

,Unnamed: 0,Module,Features,GeneName,CellType,Unnamed: 5,Unnamed: 6
0,1,BcellM10,ENSG00000204389,HSPA1A,Bcell,NaN,NaN
1,2,BcellM10,ENSG00000114021,NIT2,Bcell,NaN,NaN
2,3,BcellM10,ENSG00000126773,PCNXL4,Bcell,NaN,NaN
3,4,BcellM10,ENSG00000010404,IDS,Bcell,NaN,NaN
4,5,BcellM10,ENSG00000125637,PSD4,Bcell,NaN,NaN
...,...,...,...,...,...,...,...
25678,25679,TempusM1,ENSG00000100591,AHSA1,Tempus,NaN,NaN
25679,25680,TempusM1,ENSG00000230099,TRBV5-4,Tempus,NaN,NaN
25680,25681,TempusM1,ENSG00000089220,PEBP1,Tempus,NaN,NaN
25681,25682,TempusM1,ENSG00000132383,RPA1,Tempus,NaN,NaN


In [16]:
NKM4 = []
BcellM7 = []
MonoM6 = []
NKM13 = []
NKM9 = []

for i in range(len(Modules)):
    if Modules["Module"][i] == "NKM4":
        NKM4.append(Modules["GeneName"][i])
    if Modules["Module"][i] == "BcellM7":
        BcellM7.append(Modules["GeneName"][i])
    if Modules["Module"][i] == "MonoM6":
        MonoM6.append(Modules["GeneName"][i])
    if Modules["Module"][i] == "NKM13":
        NKM13.append(Modules["GeneName"][i])
    if Modules["Module"][i] == "NKM9":
        NKM9.append(Modules["GeneName"][i])  

In [17]:
print(len(NKM4), len(BcellM7), len(MonoM6), len(NKM13), len(NKM9))

92 777 611 202 304


## Nodes creation

In [81]:
parents = np.unique(list(Network["ParentGeneName"]))
children = np.unique(list(Network["ChildGeneName"]))

nodes = list(parents) + list(children)
nodes = np.unique(nodes)

In [144]:
nodes

array(['1-Mar', '1-Sep', '10-Sep', ..., 'hsa-mir-6723', 'mars-02',
       'snoU13'], dtype='<U18')

In [82]:
nodes_and_groups = {}

for i in nodes:
    if i in NKM4:
        nodes_and_groups[i] = "NKM4"
    elif i in BcellM7:
        nodes_and_groups[i] = "BcellM7"
    elif i in MonoM6:
        nodes_and_groups[i] = "MonoM6"
    elif i in NKM13:
        nodes_and_groups[i] = "NKM13"
    elif i in NKM9:
        nodes_and_groups[i] = "NKM9"
    else:
        nodes_and_groups[i] = "none"

In [145]:
nodes_and_groups

{'1-Mar': 'none',
 '1-Sep': 'none',
 '10-Sep': 'none',
 '15-Sep': 'none',
 '2-Mar': 'none',
 '2-Sep': 'none',
 '3-Mar': 'none',
 '4-Sep': 'none',
 '5-Mar': 'none',
 '5-Sep': 'none',
 '6-Mar': 'none',
 '6-Sep': 'none',
 '61E3.4': 'none',
 '7-Mar': 'none',
 '7-Sep': 'none',
 '8-Mar': 'none',
 '8-Sep': 'none',
 '9-Mar': 'none',
 '9-Sep': 'none',
 'A1BG-AS1': 'none',
 'A2M': 'none',
 'A2MP1': 'none',
 'AAAS': 'none',
 'AACS': 'none',
 'AAED1': 'none',
 'AAGAB': 'none',
 'AAK1': 'none',
 'AAMDC': 'none',
 'AAMP': 'none',
 'AANAT': 'none',
 'AAR2': 'none',
 'AARS': 'NKM9',
 'AARS2': 'none',
 'AARSD1': 'none',
 'AASDH': 'none',
 'AASDHPPT': 'none',
 'AASS': 'none',
 'AATF': 'none',
 'AATK': 'none',
 'ABAT': 'none',
 'ABC7-42389800N19.1': 'MonoM6',
 'ABCA1': 'none',
 'ABCA10': 'none',
 'ABCA13': 'none',
 'ABCA2': 'none',
 'ABCA3': 'none',
 'ABCA5': 'none',
 'ABCB1': 'none',
 'ABCB6': 'none',
 'ABCB7': 'none',
 'ABCB8': 'none',
 'ABCB9': 'none',
 'ABCC1': 'none',
 'ABCC10': 'none',
 'ABCC13': '

## Links creation

In [83]:
links = []
for parent, child in zip(Network["ParentGeneName"], Network["ChildGeneName"]):
    links.append([parent, child])

In [84]:
final = {"nodes": [], "links": []}

for node in nodes_and_groups:
    final["nodes"] += [{"id": node, "group": nodes_and_groups[node]}]

for parent, child in links:
    final["links"] += [{"source": parent, "target": child, "value": 1}]

In [85]:
with open('complete_data.json', 'w') as fp:
    json.dump(final, fp)

________

# Modules

### BcellM7

In [105]:
BcellM7_KD = {}
for kd, gene in zip(KDs["BcellM7_KD"], KDs["geneName_KD"]):
    BcellM7_KD[gene] = kd

len(BcellM7_KD), len(nodes)

(904, 13306)

In [142]:
links


[['GCLC', 'PSME4'],
 ['LAS1L', 'NSUN2'],
 ['LASP1', 'RAC2'],
 ['LASP1', 'RASSF5'],
 ['LASP1', 'ARF3'],
 ['LASP1', 'SPRED2'],
 ['LASP1', 'ACTR10'],
 ['M6PR', 'PDIA6'],
 ['CFLAR', 'KIAA0556'],
 ['CFLAR', 'Metazoa_SRP'],
 ['CFLAR', 'AC007283.5'],
 ['CFLAR', 'SLC16A1'],
 ['RBM6', 'REV1'],
 ['FAM214B', 'C17orf103'],
 ['FAM214B', 'WDR45'],
 ['FBXL3', 'DR1'],
 ['FBXL3', 'LCOR'],
 ['FBXL3', 'FOXN2'],
 ['SYNRG', 'DCAF5'],
 ['SYNRG', 'ZBED2'],
 ['SYNRG', 'FAM115C'],
 ['SYNRG', 'ABL2'],
 ['DBF4', 'FKBP3'],
 ['PAFAH1B1', 'ZNF398'],
 ['MPND', 'PRR22'],
 ['IL32', 'CD2'],
 ['RPS20', 'RP11-85K15.2'],
 ['RPS20', 'GAS5'],
 ['RPS20', 'MRPL32'],
 ['RPS20', 'SNN'],
 ['RPS20', 'RPS8'],
 ['CSDE1', 'TERF2IP'],
 ['TRAF3IP3', 'KIAA0907'],
 ['TRAF3IP3', 'GLUL'],
 ['TRAF3IP3', 'MINOS1'],
 ['TRAF3IP3', 'ZNF587B'],
 ['TRAF3IP3', 'AC013264.2'],
 ['TRAF3IP3', 'BCS1L'],
 ['TRAF3IP3', 'ACYP2'],
 ['TRAF3IP3', 'ACRC'],
 ['IFFO1', 'SDF4'],
 ['IFFO1', 'KCNK6'],
 ['IFFO1', 'CTB-55O6.12'],
 ['IFFO1', 'BRSK2'],
 ['STAB1', 'AS

In [114]:
## THERE IS NOT INFORMATION ABOUT KD FOR ALL NODES. I SET THEM TO 0 IF NON EXISTENT IN THE KD DOCUMENT

for n in nodes:
    if n not in BcellM7_KD:
        BcellM7_KD[n] = 0

len(BcellM7_KD), len(nodes)

(13308, 13306)

In [143]:
nodes_and_groups

{'1-Mar': 'none',
 '1-Sep': 'none',
 '10-Sep': 'none',
 '15-Sep': 'none',
 '2-Mar': 'none',
 '2-Sep': 'none',
 '3-Mar': 'none',
 '4-Sep': 'none',
 '5-Mar': 'none',
 '5-Sep': 'none',
 '6-Mar': 'none',
 '6-Sep': 'none',
 '61E3.4': 'none',
 '7-Mar': 'none',
 '7-Sep': 'none',
 '8-Mar': 'none',
 '8-Sep': 'none',
 '9-Mar': 'none',
 '9-Sep': 'none',
 'A1BG-AS1': 'none',
 'A2M': 'none',
 'A2MP1': 'none',
 'AAAS': 'none',
 'AACS': 'none',
 'AAED1': 'none',
 'AAGAB': 'none',
 'AAK1': 'none',
 'AAMDC': 'none',
 'AAMP': 'none',
 'AANAT': 'none',
 'AAR2': 'none',
 'AARS': 'NKM9',
 'AARS2': 'none',
 'AARSD1': 'none',
 'AASDH': 'none',
 'AASDHPPT': 'none',
 'AASS': 'none',
 'AATF': 'none',
 'AATK': 'none',
 'ABAT': 'none',
 'ABC7-42389800N19.1': 'MonoM6',
 'ABCA1': 'none',
 'ABCA10': 'none',
 'ABCA13': 'none',
 'ABCA2': 'none',
 'ABCA3': 'none',
 'ABCA5': 'none',
 'ABCB1': 'none',
 'ABCB6': 'none',
 'ABCB7': 'none',
 'ABCB8': 'none',
 'ABCB9': 'none',
 'ABCC1': 'none',
 'ABCC10': 'none',
 'ABCC13': '

In [182]:
links_BcellM7 = []
nodes_BcellM7 = []

for parent, child in links:
    if nodes_and_groups[parent] == "BcellM7" or nodes_and_groups[child] == "BcellM7":
        links_BcellM7.append([parent, child])
        nodes_BcellM7.append([parent, nodes_and_groups[parent], BcellM7_KD[parent]])
        nodes_BcellM7.append([child, nodes_and_groups[child], BcellM7_KD[child]]) 


In [183]:
nodes_BcellM7 = set(map(tuple, nodes_BcellM7))

In [184]:
BcellM7_final = {"nodes": [], "links": []}

for node, group, kd in nodes_BcellM7:
    BcellM7_final["nodes"] += [{"id": node, "group": group, "kd": kd}]

for parent, child in links_BcellM7:
    BcellM7_final["links"] += [{"source": parent, "target": child, "value": 1}]

In [185]:
BcellM7_final

{'nodes': [{'id': 'AOAH', 'group': 'BcellM7', 'kd': 0},
  {'id': 'CTSH', 'group': 'none', 'kd': 0},
  {'id': 'OCIAD2', 'group': 'none', 'kd': 0},
  {'id': 'LARP1', 'group': 'none', 'kd': 0},
  {'id': 'SCIMP', 'group': 'none', 'kd': 0},
  {'id': 'CHMP2A', 'group': 'none', 'kd': 0},
  {'id': 'MT-CO1', 'group': 'BcellM7', 'kd': 0},
  {'id': 'PGF', 'group': 'none', 'kd': 0},
  {'id': 'APOBEC3A', 'group': 'BcellM7', 'kd': 0},
  {'id': 'RP11-848P1.9', 'group': 'MonoM6', 'kd': 0},
  {'id': 'ATP8B3', 'group': 'BcellM7', 'kd': 0},
  {'id': 'CLYBL', 'group': 'MonoM6', 'kd': 0},
  {'id': 'SLC27A1', 'group': 'BcellM7', 'kd': 0},
  {'id': 'PTPN6', 'group': 'none', 'kd': 0},
  {'id': 'RNF149', 'group': 'BcellM7', 'kd': 0},
  {'id': 'ZDHHC1', 'group': 'none', 'kd': 0},
  {'id': 'NPR2', 'group': 'none', 'kd': 0},
  {'id': 'CNPY3', 'group': 'none', 'kd': 0},
  {'id': 'CARD11', 'group': 'none', 'kd': 0},
  {'id': 'TREML1', 'group': 'NKM13', 'kd': 0},
  {'id': 'INHBB', 'group': 'none', 'kd': 0},
  {'id':

In [118]:
with open('BcellM7_final.json', 'w') as fp:
    json.dump(BcellM7_final, fp)

### MonoM6


In [120]:
MonoM6_KD = {}
for kd, gene in zip(KDs["MonoM6_KD"], KDs["geneName_KD"]):
    MonoM6_KD[gene] = kd

len(MonoM6_KD), len(nodes)

(904, 13306)

In [121]:
## THERE IS NOT INFORMATION ABOUT KD FOR ALL NODES. I SET THEM TO 0 IF NON EXISTENT IN THE KD DOCUMENT

for n in nodes:
    if n not in MonoM6_KD:
        MonoM6_KD[n] = 0

len(MonoM6_KD), len(nodes)

(13308, 13306)

In [146]:
links

[['GCLC', 'PSME4'],
 ['LAS1L', 'NSUN2'],
 ['LASP1', 'RAC2'],
 ['LASP1', 'RASSF5'],
 ['LASP1', 'ARF3'],
 ['LASP1', 'SPRED2'],
 ['LASP1', 'ACTR10'],
 ['M6PR', 'PDIA6'],
 ['CFLAR', 'KIAA0556'],
 ['CFLAR', 'Metazoa_SRP'],
 ['CFLAR', 'AC007283.5'],
 ['CFLAR', 'SLC16A1'],
 ['RBM6', 'REV1'],
 ['FAM214B', 'C17orf103'],
 ['FAM214B', 'WDR45'],
 ['FBXL3', 'DR1'],
 ['FBXL3', 'LCOR'],
 ['FBXL3', 'FOXN2'],
 ['SYNRG', 'DCAF5'],
 ['SYNRG', 'ZBED2'],
 ['SYNRG', 'FAM115C'],
 ['SYNRG', 'ABL2'],
 ['DBF4', 'FKBP3'],
 ['PAFAH1B1', 'ZNF398'],
 ['MPND', 'PRR22'],
 ['IL32', 'CD2'],
 ['RPS20', 'RP11-85K15.2'],
 ['RPS20', 'GAS5'],
 ['RPS20', 'MRPL32'],
 ['RPS20', 'SNN'],
 ['RPS20', 'RPS8'],
 ['CSDE1', 'TERF2IP'],
 ['TRAF3IP3', 'KIAA0907'],
 ['TRAF3IP3', 'GLUL'],
 ['TRAF3IP3', 'MINOS1'],
 ['TRAF3IP3', 'ZNF587B'],
 ['TRAF3IP3', 'AC013264.2'],
 ['TRAF3IP3', 'BCS1L'],
 ['TRAF3IP3', 'ACYP2'],
 ['TRAF3IP3', 'ACRC'],
 ['IFFO1', 'SDF4'],
 ['IFFO1', 'KCNK6'],
 ['IFFO1', 'CTB-55O6.12'],
 ['IFFO1', 'BRSK2'],
 ['STAB1', 'AS

In [122]:
links_MonoM6 = []
nodes_MonoM6 = []

for parent, child in links:
    if nodes_and_groups[parent] == "MonoM6" or nodes_and_groups[child] == "MonoM6":
        links_MonoM6.append([parent, child])
        nodes_MonoM6.append([parent, nodes_and_groups[parent], MonoM6_KD[parent]])
        nodes_MonoM6.append([child, nodes_and_groups[child], MonoM6_KD[child]])      

In [124]:
MonoM6_final = {"nodes": [], "links": []}

for node, group, kd in nodes_MonoM6:
    MonoM6_final["nodes"] += [{"id": node, "group": group, "kd": kd}]

for parent, child in links_MonoM6:
    MonoM6_final["links"] += [{"source": parent, "target": child, "value": 1}]

In [125]:
with open('MonoM6_final.json', 'w') as fp:
    json.dump(MonoM6_final, fp)

### NKM13

In [129]:
NKM13_KD = {}
for kd, gene in zip(KDs["NKM13_KD"], KDs["geneName_KD"]):
    NKM13_KD[gene] = kd

len(NKM13_KD), len(nodes)

(904, 13306)

In [130]:
## THERE IS NOT INFORMATION ABOUT KD FOR ALL NODES. I SET THEM TO 0 IF NON EXISTENT IN THE KD DOCUMENT

for n in nodes:
    if n not in NKM13_KD:
        NKM13_KD[n] = 0

len(NKM13_KD), len(nodes)

(13308, 13306)

In [131]:
links_NKM13 = []
nodes_NKM13 = []

for parent, child in links:
    if nodes_and_groups[parent] == "NKM13" or nodes_and_groups[child] == "NKM13":
        links_NKM13.append([parent, child])
        nodes_NKM13.append([parent, nodes_and_groups[parent], NKM13_KD[parent]])
        nodes_NKM13.append([child, nodes_and_groups[child], NKM13_KD[child]])      

In [132]:
NKM13_final = {"nodes": [], "links": []}

for node, group, kd in nodes_NKM13:
    NKM13_final["nodes"] += [{"id": node, "group": group, "kd": kd}]

for parent, child in links_NKM13:
    NKM13_final["links"] += [{"source": parent, "target": child, "value": 1}]

In [133]:
with open('NKM13_final.json', 'w') as fp:
    json.dump(NKM13_final, fp)

### NKM9 - Not modified yet

In [129]:
NKM9_KD = {}
for kd, gene in zip(KDs["NKM13_KD"], KDs["geneName_KD"]):
    NKM13_KD[gene] = kd

len(NKM13_KD), len(nodes)

(904, 13306)

In [130]:
## THERE IS NOT INFORMATION ABOUT KD FOR ALL NODES. I SET THEM TO 0 IF NON EXISTENT IN THE KD DOCUMENT

for n in nodes:
    if n not in NKM13_KD:
        NKM13_KD[n] = 0

len(NKM13_KD), len(nodes)

(13308, 13306)

In [131]:
links_NKM13 = []
nodes_NKM13 = []

for parent, child in links:
    if nodes_and_groups[parent] == "NKM13" or nodes_and_groups[child] == "NKM13":
        links_NKM13.append([parent, child])
        nodes_NKM13.append([parent, nodes_and_groups[parent], NKM13_KD[parent]])
        nodes_NKM13.append([child, nodes_and_groups[child], NKM13_KD[child]])      

In [132]:
NKM13_final = {"nodes": [], "links": []}

for node, group, kd in nodes_NKM13:
    NKM13_final["nodes"] += [{"id": node, "group": group, "kd": kd}]

for parent, child in links_NKM13:
    NKM13_final["links"] += [{"source": parent, "target": child, "value": 1}]

In [133]:
with open('NKM13_final.json', 'w') as fp:
    json.dump(NKM13_final, fp)